In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.feature_extraction.image import extract_patches_2d
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from cuml.decomposition import PCA  # 替换 sklearn 的 PCA

from tqdm import tqdm

import cv2  

# import torchvision.transforms as transforms

import matplotlib.pyplot as plt

# import cudf
from cuml.ensemble import RandomForestClassifier as cuRF
from cuml.metrics import accuracy_score as cu_accuracy_score
from cuml.svm import SVC
from cuml.metrics import accuracy_score
import time

csv_path = "../../data/train.csv"
img_dir_eog = "../../data/train_eog_ims"
img_dir_hog = "../../data/train_hog_ims"
img_dir_sift = "../../data/train_sift_ims"
data = pd.read_csv(csv_path)

# transform = transforms.Compose([
#     transforms.RandomResizedCrop(224),  # 随机裁剪并调整大小
#     transforms.RandomHorizontalFlip(),  # 随机水平翻转
#     transforms.RandomRotation(15),     # 随机旋转
#     # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)  # 颜色抖动
#     # transforms.ToTensor(),             # 转换为张量
#     # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 标准化
# ])

X = []
y = []

for _, row in tqdm(data.iterrows(), total=len(data), desc="Processing images"):
    img_sift_path = os.path.join(img_dir_sift, row.iloc[0])
    img_hog_path = os.path.join(img_dir_hog, row.iloc[0])
    img_eog_path = os.path.join(img_dir_eog, row.iloc[0])
    
    label = int(row.iloc[1])
    
    img_sift = Image.open(img_sift_path).convert("L")  # 转换为灰度图像
    img_hog = Image.open(img_hog_path).convert("L")  # 转换为灰度图像
    img_eog = Image.open(img_eog_path).convert("L")  # 转换为灰度图像
    
    
    # img_resized = img.resize((128, 128), Image.Resampling.LANCZOS)

    # img = transform(img)
    # img = np.array(img).flatten()
    
    # img = cv2.imread(img_path)
    # img = cv2.cvtColor(img, cv2.GRA)  # 转换为 RGB 格式
    img_sift = np.array(img_sift)  # 转换为 NumPy 数组
    img_hog = np.array(img_hog)  # 转换为 NumPy 数组
    img_eog = np.array(img_eog)  # 转换为 NumPy 数组
    
    img_resized_sift = cv2.resize(img_sift, (64, 64), interpolation=cv2.INTER_CUBIC)
    img_resized_hog = cv2.resize(img_hog, (128, 128), interpolation=cv2.INTER_CUBIC)
    img_resized_eog = cv2.resize(img_eog, (64, 64), interpolation=cv2.INTER_CUBIC)
    
    img_flattened = img_resized_hog.flatten()
    img_flattened = np.concatenate((img_flattened, img_resized_sift.flatten()))
    img_flattened = np.concatenate((img_flattened, img_resized_eog.flatten()))


    X.append(img_flattened)
    y.append(label)

X = np.array(X)
y = np.array(y)


Processing images: 100%|██████████| 50000/50000 [00:20<00:00, 2486.43it/s]


In [2]:
print(X.shape)
print(max(X[0]))

(50000, 24576)
255


In [3]:
import torch
torch.cuda.empty_cache()

In [4]:
best_params = None
best_accuracy = 0

X = X.astype(np.float32)

pca_dims = [175, 185, 200]
Cs = [3, 5, 7, 10]
gamma = ["scale"]
np.random.seed(42)
for dim in tqdm(pca_dims, total=len(pca_dims), desc="PCA dimensions"):
    if(dim!=-1):
        pca = PCA(n_components=dim)
        X_pca = pca.fit_transform(X)
    else:
        X_pca = float(X)
        print(f"Reduced to {dim} dimensions")
    for C in Cs:
        for g in gamma:
            print(f"dim: {dim}, C: {C}")
            start_time = time.time()
            X_train, X_test, Y_train, Y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)
            # 使用 cuML 的 SVC 模型进行训练
            svm_model = SVC(kernel='rbf', C=C, gamma=g)
            svm_model.fit(X_train, Y_train)
            end_time = time.time()
            print(f"Train-test split time: {end_time - start_time:.4f} seconds")
            Y_pred = svm_model.predict(X_test)
            # 计算准确率
            acc = accuracy_score(Y_test, Y_pred)
            print("Accuracy:", acc)
            
            if acc > best_accuracy:
                best_accuracy = acc
                best_params = {"dim": dim, "C": C, "gamma": g}

print("Best Parameters:", best_params)
print("Best Accuracy:", best_accuracy)

# Best Parameters: {'dim': 100, 'C': 10}
# Best Accuracy: 0.5564000010490417

PCA dimensions:   0%|          | 0/3 [00:00<?, ?it/s]

dim: 175, C: 3
Train-test split time: 2.8052 seconds
Accuracy: 0.6485999822616577
dim: 175, C: 5
Train-test split time: 3.2760 seconds
Accuracy: 0.6500999927520752
dim: 175, C: 7
Train-test split time: 3.6859 seconds
Accuracy: 0.6499000191688538
dim: 175, C: 10
Train-test split time: 4.0267 seconds


PCA dimensions:  33%|███▎      | 1/3 [00:39<01:19, 39.69s/it]

Accuracy: 0.6481999754905701
dim: 185, C: 3
Train-test split time: 2.7917 seconds
Accuracy: 0.6507999897003174
dim: 185, C: 5
Train-test split time: 3.3521 seconds
Accuracy: 0.6527000069618225
dim: 185, C: 7
Train-test split time: 3.6898 seconds
Accuracy: 0.6536999940872192
dim: 185, C: 10
Train-test split time: 3.9824 seconds


PCA dimensions:  67%|██████▋   | 2/3 [01:17<00:38, 38.59s/it]

Accuracy: 0.6496999859809875
dim: 200, C: 3
Train-test split time: 2.8356 seconds
Accuracy: 0.650600016117096
dim: 200, C: 5
Train-test split time: 3.3468 seconds
Accuracy: 0.6549999713897705
dim: 200, C: 7
Train-test split time: 3.6731 seconds
Accuracy: 0.6520000100135803
dim: 200, C: 10
Train-test split time: 3.9798 seconds


PCA dimensions: 100%|██████████| 3/3 [01:55<00:00, 38.52s/it]

Accuracy: 0.6488999724388123
Best Parameters: {'dim': 200, 'C': 5, 'gamma': 'scale'}
Best Accuracy: 0.6549999713897705
